# Comparison between different solutions

In this notebook we'll compare all the solutions that were tested before deciding on the final one.

## Installations and imports

In [1]:
!pip install haversine ortools pulp numba

In [2]:
import pandas as pd
from Simulation import Simulation
from GeneticAlgorithmGraph import GeneticAlgorithmGraph as GAGraph
from GoogleRouting import GoogleRouting
from AdjustedKMeans import AdjustedKMeans

## Data loading and cleaning

In [3]:
FILE_PATH = 'dataset.csv'

Removal of useless columns and records.

In [4]:
df = pd.read_csv(FILE_PATH)
df['detected_at'] = pd.to_datetime(df['detected_at'])

USELESS_COLUMNS = ['id','raw_data_id','raw_data_setting_id','Seriale','created_at','DataUltimaRilevazione','DataUltimaTrasmissione','DataPrimaInstallazione','Indirizzo','Cap','UnitaTerritoriale','Viario','Tronco', 'Esterno', 'AreaGestionale']
df.drop(USELESS_COLUMNS, axis=1, inplace=True)

df.occluded.replace({1:False, 2:True}, inplace=True)
df.fillna(value=False, inplace=True)
df = df[df.TipoAnomalia == False]
df.drop('TipoAnomalia', axis=1, inplace=True)
df.rename(columns={'Latitudine': 'latitude', 'Longitudine':'longitude'}, inplace=True)

df.set_index('detected_at', inplace=True, drop=True)
df.sort_index(inplace=True)

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (22) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


Remaining columns:

In [5]:
print(df.columns.values[1:])

['bin_serial' 'bin_level' 'occluded' 'latitude' 'longitude']


## Functions

Slender distance:

In [6]:
from haversine import haversine
import math

def slender_distance(p1, p2, center, alpha_1=1, alpha_2=0):
    ang_d = math.radians(get_angle(p1, p2, center))
    radial_d = haversine(p1, p2)
    return alpha_1*ang_d+alpha_2*radial_d

def get_angle(a, b, origin):
    ang = math.degrees(math.atan2(b[1]-origin[1], b[0]-origin[0]) - math.atan2(a[1]-origin[1], a[0]-origin[0]))
    ang = abs(ang) if abs(ang) < 180 else 360-abs(ang)
    return ang

## Test

### Test configuration

In [7]:
def filter_function(data, level=3):
    new_data = data.drop_duplicates(subset='bin_serial', keep='last')
    new_data = new_data[(new_data.bin_level > level) | new_data.occluded]
    return new_data

In [8]:
import warnings
warnings.filterwarnings('ignore')

start_date = '2019-09-01 00:00:00'
end_date = '2019-10-01 00:00:00'
data = df[start_date : end_date]

depot = (45.5069182, 9.2684501)
vehicle_capacities = 200
num_vehicles = 20

In [9]:
def run_simulation(config, name):
    simulation = Simulation(depot, config, window_size=6, max_size=200, filter_function=filter_function, filter_kwargs={})
    routes = simulation.compute_simulation(data, pd.to_datetime(start_date), pd.to_datetime(end_date), speed=30, emp_time=60, debug=False)
    simulation.to_csv(f'Output\\{name}.csv')
    score = simulation.get_score()

    print(f'# {name} #')
    print(f'Numero di turni eseguiti: {str(len(routes))}.')
    print(f'Distanza totale: {str(score)} km.')
    total_bins = sum([len(routes[w][c]) for w in range(len(routes)) for c in range(len(routes[w]))])
    print(f'Numero di cestini svuotati: {str(total_bins)}.')
    total_vehs = sum([len(routes[w]) for w in range(len(routes))])
    print(f'Numero di veicoli usati: {str(total_vehs)}.')

### Baseline: Standard k-Means + Christofides

In [10]:
kmeans_kwargs = {
    'max_size' : 200,
    'balanced': False,
    'distance': lambda p1, p2 : slender_distance(p1, p2, depot, 0, 1),
}

routing_kwargs = {
    'distance_function': haversine,
    'vehicle_capacities': 200,
    'num_vehicles': 1,
}

baseline_config = {
  'cluster_class': AdjustedKMeans,
  'cluster_kwargs': kmeans_kwargs,
  'graph_class': GoogleRouting,
  'graph_kwargs': routing_kwargs,
}

run_simulation(baseline_config, 'Baseline')

# Baseline #
Numero di turni eseguiti: 120.
Distanza totale: 19222.724620000008 km.
Numero di cestini svuotati: 130845.
Numero di veicoli usati: 709.


### Google Routing

In [11]:
GR_kwargs = {
    'distance_function': haversine,
    'vehicle_capacities': 200,
    'num_vehicles': 20,
}

GR_config = {
  'cluster_class': None,
  'cluster_kwargs': {},
  'graph_class': GoogleRouting,
  'graph_kwargs': GR_kwargs,
}

run_simulation(GR_config, 'GoogleRouting')

# GoogleRouting #
Numero di turni eseguiti: 120.
Distanza totale: 18661.584350000012 km.
Numero di cestini svuotati: 130845.
Numero di veicoli usati: 709.


### SC2G - Shape-Controlled Clustering + Genetic algorithm

In [12]:
clustering_kwargs = {
    'max_size' : 200,
    'balanced': True,
    'distance': lambda p1, p2 : slender_distance(p1, p2, depot, 1, 0),
}

# GA settings
pop_size = 500
elite_size = int(0.05*pop_size)
mutation_rate = 0.1
generations = 200

GA_kwargs = {
    'pop_size': pop_size,
    'elite_size': elite_size, 
    'mutation_rate': mutation_rate,
    'generations': generations,
    'metric': 'km',
}

SC2G_config = {
  'cluster_class': AdjustedKMeans,
  'cluster_kwargs': clustering_kwargs,
  'graph_class': GAGraph,
  'graph_kwargs': GA_kwargs,
}

run_simulation(SC2G_config, 'SC2G')

# SC2G #
Numero di turni eseguiti: 120.
Distanza totale: 18706.83744 km.
Numero di cestini svuotati: 130845.
Numero di veicoli usati: 709.


### SC3 balanced

In [13]:
clustering_balanced = {
    'max_size' : 200,
    'balanced': True,
    'distance': lambda p1, p2 : slender_distance(p1, p2, depot, 1, 0),
}

routing_kwargs = {
    'distance_function': haversine,
    'vehicle_capacities': 200,
    'num_vehicles': 1,
}

SC3_balanced_config = {
  'cluster_class': AdjustedKMeans,
  'cluster_kwargs': clustering_kwargs,
  'graph_class': GoogleRouting,
  'graph_kwargs': routing_kwargs,
}

run_simulation(SC3_balanced_config, 'SC3Balanced')

# SC3Balanced #
Numero di turni eseguiti: 120.
Distanza totale: 17790.80992 km.
Numero di cestini svuotati: 130845.
Numero di veicoli usati: 709.
